# Task4 Object Tracking

In [1]:
# Libraries
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

In [12]:
# Load the YOLO11 model
model = YOLO("yolo11n.pt")

# Open the video file
video_path = "tracking.mp4"
cap = cv2.VideoCapture(video_path)

# storing the tracking history
track_history = defaultdict(list)

# frames to capture
selected_frame = [800] 
frame_counter = 0
prev_frame = None

# Looping
while cap.isOpened():
    # reading the frame
    success, frame = cap.read()

    if success:
        frame_counter += 1

        # running the model
        results = model.track(frame, persist=True)

        # box ids and track ids
        boxes = results[0].boxes.xywh.cpu().numpy()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Annotating the path
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            center = (int(x), int(y))  # center for the box
            track_history[track_id].append(center)  # adding to path history

            # plotting the tracking line
            if len(track_history[track_id]) > 1:
                for i in range(1, len(track_history[track_id])):
                    cv2.line(frame,
                             track_history[track_id][i - 1],
                             track_history[track_id][i],
                             color=(0, 255, 0),
                             thickness=2)

            # bounding box
            cv2.rectangle(frame,
                          (int(x - w / 2), int(y - h / 2)),
                          (int(x + w / 2), int(y + h / 2)),
                          color=(255, 0, 0),
                          thickness=2)

            # ID
            cv2.putText(frame, f"ID: {track_id}", (int(x), int(y - h / 2 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

            cv2.putText(frame, f"Frame: {frame_counter}", (10, 30),  # Top-left corner
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)  # Red text
            
        if frame_counter in selected_frame and prev_frame is not None:
            print(f"Reached selected frame: {frame_counter}")

            # previous and current frames
            cv2.imshow("Previous Frame", prev_frame)
            cv2.imshow("Current Frame", frame)

           
            key = cv2.waitKey(0) & 0xFF
            if key == ord('s'):  # s to save
                cv2.imwrite(f"frame_{frame_counter - 1}.jpg", prev_frame)
                cv2.imwrite(f"frame_{frame_counter}.jpg", frame)
                print(f"Saved frames: frame_{frame_counter - 1}.jpg and frame_{frame_counter}.jpg")
            elif key == ord('q'): # q to exit
                break

        # display the path
        cv2.imshow("tracking with path", frame)

        # Update
        prev_frame = frame.copy()
        
        #loop brek
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break


cap.release()
cv2.destroyAllWindows()



0: 384x640 2 persons, 6 cars, 1 truck, 55.0ms
Speed: 2.0ms preprocess, 55.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 6 cars, 1 truck, 69.0ms
Speed: 2.0ms preprocess, 69.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 6 cars, 1 truck, 62.0ms
Speed: 1.0ms preprocess, 62.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 6 cars, 1 truck, 66.0ms
Speed: 1.0ms preprocess, 66.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 6 cars, 2 trucks, 67.0ms
Speed: 1.0ms preprocess, 67.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 6 cars, 2 trucks, 65.0ms
Speed: 2.0ms preprocess, 65.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 5 cars, 1 truck, 56.0ms
Speed: 1.0ms preprocess, 56.0ms inference, 1.0ms postprocess per image at shape (1, 3, 